In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.datasets.fashion_mnist import load_data as load_data_Fashion_MNIST
from tensorflow.keras.datasets.cifar10 import load_data as load_data_CIFAR_10

from nn.losses import *
from nn.layers import *
from nn.initializers import *
from nn.activations import *
from nn.network import NeuralNetwork, MulticlassNNClassifier

In [3]:
%load_ext autoreload

In [4]:
%autoreload

## Fashion MNIST

### Loading data

In [22]:
(x_train_fmnist, y_train_fmnist), (x_test_fmnist, y_test_fmnist) = load_data_Fashion_MNIST()

fmnist = {
    'train': {
        'x': x_train_fmnist,
        'y': y_train_fmnist
    },
    'test': {
        'x': x_test_fmnist,
        'y': y_test_fmnist
    }
}

### Tensorflow

### This library

## CIFAR 10

### Loading data

In [23]:
(x_train_cifar10, y_train_cifar10), (x_test_cifar10, y_test_cifar10) = load_data_CIFAR_10()

cifar10 = {
    'train': {
        'x': x_train_cifar10,
        'y': y_train_cifar10
    },
    'test': {
        'x': x_test_cifar10,
        'y': y_test_cifar10
    }
}

### Tensorflow

### This library

## 20 class dataset

### Loading data

In [24]:
ds = tf.keras.utils.image_dataset_from_directory(
    'data/cars',
    label_mode='int',
    shuffle=True,
    seed=42,
    image_size=(224, 224),
)

x_cars, y_cars = zip(*ds)
x_cars = np.concatenate(x_cars)
y_cars = np.concatenate(y_cars)

x_train_cars, x_test_cars, y_train_cars, y_test_cars = train_test_split(
    x_cars,
    y_cars,
    test_size=0.20,
    random_state=42
)

cars = {
    'train': {
        'x': x_train_cars,
        'y': y_train_cars
    },
    'test': {
        'x': x_test_cars,
        'y': y_test_cars
    }
}

Found 3235 files belonging to 20 classes.


### Tensorflow

### This library